set [SDC](https://commons.wikimedia.org/wiki/Commons:Structured_data#:~:text=Structured%20data%20on%20Commons%20is,from%20Wikidata%2C%20Wikimedia's%20knowledge%20base.) depicts from files uploaded in [spa2Commons](https://commons.wikimedia.org/wiki/Category:Uploaded_with_spa2Commons)

### Theory depicts
* all pictures from [SPA](https://portrattarkiv.se/about) depicts one person
* one category in WIkicommons is connected to the person that the picture depicts

#### How to - done
* a csv file is created from a pile using
  * petscan creates the pile ex. [set output pagepile](https://petscan.wmflabs.org/?psid=20485153)--> [pagepile 39223](https://pagepile.toolforge.org/api.php?action=get_data&id=39223)
  * [minefield](https://hay.toolforge.org/minefield/) creates the csv file  
    * see [gist](https://gist.github.com/salgo60/b5d05fae5c865b678edb338b09e4b302)
* [video](https://www.youtube.com/watch?v=FUoG0veIeMY&feature=youtu.be)     
    
### Theory SPA identifier
* a picture uploaded to Wikicommons using [SPA2common javascript](https://github.com/salgo60/spa2Commons) will have the link to the SPA picture in the template Information and param **Source** ex. [File:Axel_Sammuli_SPA3.jpg](https://commons.wikimedia.org/wiki/File:Axel_Sammuli_SPA3.jpg) -->  SPA id = [SPA idYB0QHyfj0hAAAAAAAAAf8g](https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAAAf8g)
#### How to
* read all pictures in category  [spa2Commons](https://commons.wikimedia.org/wiki/Category:Uploaded_with_spa2Commons) and check param **Source**
### Misc
* [SPARQL mwapi](https://en.wikibooks.org/wiki/SPARQL/SERVICE_-_mwapi)
* [API:Categories](https://www.mediawiki.org/wiki/API:Categories)
* Test SPARQL
   * [get files in Categories](https://wcqs-beta.wmflabs.org/#%23Wikidata%20items%20of%20files%20in%20Category%3AArtworks%20with%20structured%20data%20with%20redirected%20P6243%20property%0ASELECT%20%3Ffile%20%3Ftitle%20%20%3Fspa%20%0AWITH%0A%7B%0A%20%20SELECT%20%3Ffile%20%3Ftitle%0A%20%20WHERE%0A%20%20%7B%0A%20%20%20%20SERVICE%20wikibase%3Amwapi%0A%20%20%20%20%7B%0A%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Aapi%20%22Generator%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Aendpoint%20%22commons.wikimedia.org%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20mwapi%3Agcmtitle%20%22Category%3AUploaded_with_spa2Commons%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20mwapi%3Agenerator%20%22categorymembers%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20mwapi%3Agcmtype%20%22file%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20mwapi%3Agcmlimit%20%22max%22%20.%0A%20%20%20%20%20%20%3Ftitle%20wikibase%3AapiOutput%20mwapi%3Atitle%20.%0A%20%20%20%20%20%20%3Fpageid%20wikibase%3AapiOutput%20%22%40pageid%22%20.%0A%20%20%20%20%7D%0A%20%20%20%20BIND%20%28URI%28CONCAT%28%27https%3A%2F%2Fcommons.wikimedia.org%2Fentity%2FM%27%2C%20%3Fpageid%29%29%20AS%20%3Ffile%29%0A%20%20%7D%0A%7D%20AS%20%25get_files%0AWHERE%0A%7B%0A%20%20INCLUDE%20%25get_files%0A%20%20OPTIONAL%20%7B%3Ffile%20wdt%3AP4819%20%3Fspa%7D%0A%7D)
* [PAWS Example Notebooks](https://wikitech.wikimedia.org/wiki/PAWS/PAWS_examples_and_recipes)   
* [hub.toolforge.org](https://hub.toolforge.org)
* [writeSDoCfromExcel](https://github.com/KBNLwikimedia/SDoC/blob/main/writeSDoCfromExcel/WriteSDoCfromExcel_nopasswd.py)

In [8]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-10-31 08:51:34.273668


In [24]:
import requests

S = requests.Session()

URL = "https://commons.wikipedia.org/w/api.php"

def get_Category(pageName):
    PARAMS = {
        "action": "query",
        "format": "json",
        "prop": "categories",
        "titles": pageName   
    }
    #print (PARAMS)
    r = S.get(url=URL, params=PARAMS)
    data = r.json()
    # TODO dont get hidden categories
    filtercat = {
        "Category:CC-BY-4.0",
        "Category:Swedish Portrait Archive",
        "Category:Uploaded with spa2Commons",
        "Category:Template Unknown (author)",
        "Category:Images with extracted images",
        "Category:Extracted images",
        "Category:Scanned with HP Deskjet F4200",
        "Category:Pages using duplicate arguments in template calls",
        "Category:Creative Commons Attribution-Share Alike missing SDC copyright status",
        "Category:Creative Commons Attribution-Share Alike 4.0 missing SDC copyright license"
        }
    target_category = ""
    PAGES = data["query"]["pages"]
    for k, v in PAGES.items():
    #    print(k,v)
        for cat in v['categories']:
            if cat["title"] not in filtercat:
                target_category = cat["title"]
                #print("\tTarget cat" ,target_category)
    return target_category

In [20]:
common_name = "File:A_G_Ahlqvist_SPA10.jpg"
get_Category(common_name)

'Category:Alfred Gustaf Ahlqvist'

In [21]:
def getWD(commonsCategory):
    urlHub = "https://hub.toolforge.org/commons:" + commonsCategory + "?format=json&site=wd"
    #print(urlHub)
    hub = S.get(url=urlHub)
    data = hub.json()
    try:
        wd = data["destination"]["url"].replace("https://www.wikidata.org/wiki/","")    
    except:
        print("Error", data)
        wd =""
    return wd

In [22]:
test_category ="Category:Axel_Rappe_(1838%E2%80%931918)"
getWD(test_category)

'Q4830349'

### get WD object the Pictiure depict

In [23]:
# used tool xxxx to get csv files with pictures 
import csv

mid_wd_list = []
with open('Cat_2commons.csv') as csvfile:
    cat_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in cat_reader:
#        print(row)
#        print(row["mid"],get_Category(row["title"]),row["url"]) 
#        print(row["mid"],getWD(get_Category(row["title"]))) 
        mid_wd_list.append([row["mid"],getWD(get_Category(row["title"]))])
        #print(mid_wd_list)
        


Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Creative Commons Attribution-Share Alike missing SDC copyright status', 'lang': 'en'}}
Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Fredrik Mauritz von Krusenstjerna', 'lang': 'en'}}
Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Georg Stoopendaal', 'lang': 'en'}}
Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Georg Stoopendaal', 'lang': 'en'}}
Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Gustaf Bergmark', 'lang': 'en'}}
Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Gustaf Bergmark', 'lang': 'en'}}
Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Gustaf-Otto Adelborg', 'lang': 'en'}}
Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Gustaf Walfrid Carlström', 'lang': 'en'}}
Errorr {'message': 'Not Found', 'context': {'text': 'commons:Category:Gustaf Walfrid C

In [25]:
import pandas as pd
df = pd.DataFrame(mid_wd_list)
df.head(10)

,0,1
0,M110377313,Q6239681
1,M110421273,Q350301
2,M110426964,Q22977459
3,M110427336,Q22977459
4,M110465218,Q2824703
5,M110465275,Q2824703
6,M110470101,Q5581039
7,M110482614,Q18327433
8,M110494600,Q362749
9,M110776582,Q18327433


In [26]:
df.to_csv("SPACategories_Mid_WD.txt")
df.shape

(3714, 2)

In [ ]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))